In [51]:
# install libraries
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y
#!mamba install bs4==4.10.0 -y
#!mamba install html5lib==1.1 -y

# get exchange rates from dataset
#!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv


import glob
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import html5lib
import requests
import pandas as pd
import re

# gather content from webpage
html_data=requests.get("https://en.wikipedia.org/wiki/List_of_largest_banks?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01").text

# parse content with BeautifulSoup
soup=BeautifulSoup(html_data)


# define column names and create empty data farme


def extract():
    data = pd.DataFrame(columns=["Name", "Market Cap (US$ Billion)"])
    for row in soup.find_all('tbody')[3].find_all('tr'):
        col = row.find_all('td')
        if (col != []):
            name = col[1].text.strip()
            market_cap = col[2].text.strip()
            market_cap = float(re.sub("[[@*&?].*[]@*&?]", "", market_cap)) # remove references

            data = data.append({"Name":name, "Market Cap (US$ Billion)":market_cap}, ignore_index=True)
    return data

# call function extract with the json file
data_bank = extract()

# display 5 first rows
data_bank.head()



,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,400.37
1,Industrial and Commercial Bank of China,295.65
2,Bank of America,279.73
3,Wells Fargo,214.34
4,China Construction Bank,207.98


In [52]:
# read data about exchange rate
exchange_data = pd.read_csv("exchange_rates.csv",index_col = 0)

# look at first columns
print(exchange_data.head())

# find exchange rate for british pound
exchange = exchange_data[exchange_data.index=="GBP"].iloc[0,0]





        Rates
AUD  1.297088
BGN  1.608653
BRL  5.409196
CAD  1.271426
CHF  0.886083


In [53]:
# transform function

def transform(data):
    # convert market cap in $ to pound
    data['Market Cap (US$ Billion)'] = round(data['Market Cap (US$ Billion)'] * exchange, 2)
    data = data.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'})

    return data

In [39]:
# load function

def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile, index = False)

targetfile ="bank_market_cap_gbp.csv"

In [54]:
# function to write logs at each step

def log(message):
    timestamp_format ='%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt", "a") as f:
        f.write(timestamp + ',' + message + '\n' )

In [55]:
# run the ETL process

log ("ETL Job Started")

log ("Extract phase Started")

extracted_data = extract()


log("Extract phase Ended")

log("Transform phase Started")

transformed_data = transform(extracted_data)

log("Transform phase Ended")

log("Load phase Started")

load(targetfile, transformed_data)

log("Load phase Ended")

